In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/12 09:54
# @Author  : Wang Yujia
# @File    : SA_for_PT_one_for_all.ipynb
# @Description : for all settings and data, just infer one params

# 0. What for
1. 给所有的data和setting，只infer一组params

# 1. Preparations
## 1.1 全局设置

In [ ]:
# small fix from here
# data path
data_selected_path = "../../data/SA_PT/datawithnp_PT_selected.csv"
# data_key path
data_key_path = "../../data/SA_PT/data_key_PT.csv"

# output path
params_opitim_path = "../../data/SA_PT/params_opitim.csv"
params_opitim_oneforall_path = "../../data/SA_PT/params_opitim_oneforall.csv"

features_GT = ['bidincrement','bidfee','retail']
# for SA
# initial params
table_5_M = [0.025,3.72]
# table_5_M = [0,2]
# lower/ upper bound
lb = [-0.3,0.01]
ub = [0.3,18]

import numpy as np
import csv
import pandas as pd
from visdom import Visdom
from SA_modified import SABoltzmann
from sko.tools import set_run_mode
from SA_for_PT_funcs_delta_eq1 import *

# viz = Visdom(env='P',use_incoming_socket=False)

## 1.2 data 读取
1. 保存data_key
2. 并且提取成功infer的data`params_all`

In [ ]:
data = pd.read_csv(data_selected_path, encoding="utf-8")
data_key = data[features_GT].copy()
data_key.drop_duplicates(inplace=True,ignore_index=True)
data_key.to_csv(data_key_path,header=True, encoding="utf-8",index=False)

# 有`N_uniq_auction`组setting
N_uniq_auction= data_key.shape[0]

print("For PT model, there are *{}* settings waiting to be inferred.".format(N_uniq_auction))

## 1.3 functions about 'key'

In [ ]:
# get key from i in 'data_key'
def get_key_from_index(i,str="NotStr"):
    if(str=="str"):
        key_i = list(data_key.iloc[i,:])
        key_i_str = (str(key_i[0]),str(key_i[1]),str(key_i[2]))
        return key_i_str
    else:
        key_i = data_key.iloc[i,:]
        return key_i

#features_GT = ['bidincrement','bidfee','retail']
def select_data_fromkey(key_i_str):
    return data[(data['bidincrement'] == key_i_str[0]) & (data['bidfee'] == key_i_str[1]) & (data['retail'] == key_i_str[2])].copy()

# 2. Equi. condition in PT model
1. 根据Eq(6)
2. 注意分辨怎么代入上面的公式
3. `delta = 1`时，公式可以大大化简，见ipad上的公式

In [ ]:

def f_Equi(t,v,d,b,alpha,labda):

    tmp = v-d*t-C(t-1,b) - b

    if (tmp>=0):
        root = (labda*f(C(t-1,b),alpha) + f(tmp,alpha)) / (labda*f(C(t-1,b)+b,alpha) + f(tmp,alpha))
        # if(np.isclose(root,0.0)):
        #     print(f"t:{t} ---- u = 0.0:{root} ---- alpha : {alpha}")
    else:
        # print("tmp starts to < 0", t)
        root = (f(C(t-1,b),alpha) - f(-tmp,alpha)) / (f(C(t-1,b)+b,alpha) + f(-tmp,alpha))
        # if(np.isclose(root,0.0)):
        #     print(f"t:{t} -- u = 0.0:{root} -- alpha : {alpha} -- lamda : {labda}")

    # if(root > 1.0):
    #     print(f"t:{t} ---- u > 1.0:{root} ---- alpha: {alpha}")

    #viz.line([[0.0,0.0]],[0],win = 'root compare',opts= dict(title='root in 2 methods'+str(t),legend=['simplify', 'sympy']))
    #viz.line([[np.float(root1),np.float(root)]],[t],win = 'root compare', update='append')

    return root

# 3. SA
## 3.1 define loss function
1. loss function: NLL for auctions with same `features_GT`
2. “one for all”的不同之处在于，累加所有setting的情况求一个loss，不再区分`data_i`

In [ ]:
def loss_func(params,other_params):
    # start_time_loss = datetime.datetime.now()
    alpha = params[0]
    # delta = 1
    labda = params[1]
    # max_T,v,d,b,T_i,cnt_n_2_i = other_params
    NLL=0.0
    for i in range(0,N_uniq_auction):

        # get i_th data_key
        key_i = get_key_from_index(i)
        # extract data with same `key_i` into a table
        data_i = select_data_fromkey(key_i)
        data_i.reset_index(drop=True,inplace=True)

        T_i = data_i['N'].astype(int)          # auction duration sequence
        max_T = int(max(T_i))                  # max duration value

        cnt_n_2_i = data_i['cnt_n_2'].astype(int)       # Number of occurrences of different durations
        # for a certain auction(like 'data_i'), 'cnt_uniq' should be all the same
        # A_i = int(data_i['cnt_uniq'].unique())
        # assert(A_i == sum(cnt_n_2_i),"'cnt_uniq' does not match with sum of 'cnt_n_2'!")

        v = float(data_i['retail'].unique())
        d = float(data_i['bidincrement'].unique())
        b = float(data_i['bidfee'].unique())
        assert max_T <= (v-b)/d, "wrong!"

        flag = 1
        # solve for U from Equi. condt.
        U_i = [0] * (max_T + 1)
        U_i[0] = 1

        for t in range(1,max_T+1):

            U_i[t] = f_Equi(t, v, d, b, alpha, labda)
            if(flag & (U_i[t]<=0)):
                # print(f"t:{t} -- u starts to <= 0.0: {U_i[t]} -- alpha : {alpha} -- lamda : {labda}")
                flag = 0

        # calculate NLL under this auction setting & PT params
        nll = 0.0
        if(U_i[0]==1):
            U_i.pop(0)
        for idx in range(0,data_i.shape[0]):
            # sum up the log prob among all durations of this auction
            # nll1 += np.cumprod(U_i)[T_i[idx]-2]*U_i[T_i[idx]-1]* cnt_n_2_i[idx]
            # NLL1 == nll
            nll += ( np.sum( np.log( U_i[0:(T_i[idx]-1)] ) ) + np.log(1-U_i[(T_i[idx]-1)]) )* cnt_n_2_i[idx]
        # print('> The loss costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time_loss).total_seconds()))
        NLL += nll
    print(f"NLL loss is {NLL}")
    return float(-NLL)

## 3.2 do SA
1. 要对所有settings做一次infer
2. 具体的：对每个setting `i`
    - 每一个setting `i` 可以提取出来一个`data_i`，代表所有auction
    - 每一个`data_i`中的`cnt_uniq`，也就是`A`，是相同的，表示setting `i` 进行的拍卖总次数.【但是这个`A`在计算loss的时候派不上用场】
    - `N`表示duration，因此paper公式里的$T_a$即`N[a]`
    - 因此有`A = sum(data_i['cnt_n_2'])`，其中的'cnt_n_2'表示了该行对应的`duration=N`发生的次数
    - 按照上文，求解`U[i]_t` which is a array with shape of (max(N)),也就是求解paper里的`p_t`
3.每次进行`L`次对参数的试探寻找，每次寻找对应一个温度一组新的参数。
    - 优化的完成/退出条件：温度小于`T_min`或者最低温度保持`max_stay_counter`次的不变
    - 鉴于温度小于`T_min`很难达到，因此基本上对一组参数进行优化要进行L*max_stay_counter+1次运算（loss运算）

In [ ]:
params_opitim = pd.DataFrame(columns=['trial_time','alpha','delta','labda','initial_loss','final_loss','avg_loss'])
max_trial = 2

In [ ]:
# Perform SA for all settings at once
for i in range(0,max_trial):
    print(f"> For the *{i}_th* trial")
    print("> Initilizing SA....... \n")
    set_run_mode(loss_func, 'cached')
    set_run_mode(loss_func, 'multithreading')

    initial_t = max(data['cnt_n_2'].astype(int))        # initial temperature
    # L=30, max_stay_counter=15
    # other_params = [max_T,v,d,b,T_i,cnt_n_2_i]
    sa_boltzmann = SABoltzmann(func=loss_func, x0=table_5_M, other_params = [],T_max=initial_t, T_min=1, learn_rate=0.2, L=20, max_stay_counter=10,
    lb=lb, ub=ub)

    print("> Now do SA....... \n")
    best_x, best_y = sa_boltzmann.run()
    # print('> The whole inference process costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time).total_seconds()))

    print("> SA ENDS....... \n")
    # if(i%10 == 0):
    #     viz.line([0.0]*(sa_boltzmann.iter_cycle+1),[0]*(sa_boltzmann.iter_cycle+1),env = "Loss of PT_2",win = 'Loss of '+str(i),opts= dict(title='Loss of '+str(i)))
    #     viz.line(np.array(sa_boltzmann.generation_best_Y),np.arange(0,sa_boltzmann.iter_cycle+1),env = "Loss of PT_2",win = 'Loss of '+str(i), update='append')

    # append the opitimized params into the df
    df_tmp = pd.DataFrame([ [i,best_x[0],1,best_x[1],sa_boltzmann.generation_best_Y[0],best_y,best_y/data.shape[0]] ],
                        columns=['trial_time','alpha','delta','labda','initial_loss','final_loss','avg_loss'])
    params_opitim = params_opitim.append(df_tmp,ignore_index=True)

In [ ]:
# save 'params_opitim' for later check
params_opitim.to_csv(params_opitim_path, header=True, encoding="utf-8",index=False)